In [5]:
! pip install datasets transformers[sentencepiece]

In [6]:
!pip install sacrebleu==1.4.14

In [7]:
from huggingface_hub import notebook_login

notebook_login()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:510: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.7.
  FutureWarning,


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [8]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [5]:
import transformers

print(transformers.__version__)

4.18.0


In [9]:
model_checkpoint = "facebook/m2m100_418M"

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#데이터셋 로딩
import pandas as pd
from datasets import Dataset
#data1 = pd.read_excel("/content/drive/MyDrive/data/1_구어체(1).xlsx")
#data2 = pd.read_excel("/content/drive/MyDrive/data/1_구어체(2).xlsx")
data3 = pd.read_excel("/content/drive/MyDrive/data/2_대화체.xlsx")

In [78]:
data1

,SID,원문,번역문
0,1,'Bible Coloring'은 성경의 아름다운 이야기를 체험 할 수 있는 컬러링 ...,Bible Coloring' is a coloring application that...
1,2,씨티은행에서 일하세요?,Do you work at a City bank?
2,3,푸리토의 베스트셀러는 해외에서 입소문만으로 4차 완판을 기록하였다.,"PURITO's bestseller, which recorded 4th rough ..."
3,4,11장에서는 예수님이 이번엔 나사로를 무덤에서 불러내어 죽은 자 가운데서 살리셨습니다.,In Chapter 11 Jesus called Lazarus from the to...
4,5,"6.5, 7, 8 사이즈가 몇 개나 더 재입고 될지 제게 알려주시면 감사하겠습니다.",I would feel grateful to know how many stocks ...
...,...,...,...
199995,199996,나는 먼저 청소기로 바닥을 밀었어요.,"First of all, I vacuumed the floor."
199996,199997,나는 먼저 팀 과제를 하고 놀러 갔어요.,I did the team assignment first and went out t...
199997,199998,나는 비 같은 멋진 연예인을 좋아해요.,I like cool entertainer like Rain.
199998,199999,나는 멋진 자연 경치를 보고 눈물을 흘렸어.,I cried seeing the amazing scenery.


In [79]:
data2

,SID,원문,번역문
0,200001.0,0 설정을 입력하고 안정될 때까지 5분 동안 기다린 후 OK 버튼을 길게 누르십시오.,"Enter into 0 setting, and wait for 5 minutes t..."
1,200002.0,0은 그들에게 아무것도 아니었지만 무는 숫자일 수가 없습니다.,The zero was nothing for them but nothing coul...
2,200003.0,"1,015버전에서 핫키 버그가 있습니다.","There is a Hotkey bug in the 1,015 version."
3,200004.0,"1,390점에서 1,440점을 득점한 사람은 재판을 위해 걸러집니다.","Individuals who got a score between 1,390 and ..."
4,200005.0,"1,400년보다 오래 전의 유적지에 있는 최초의 성당에서 숭배자들은 그것을 인지했을...","Indeed, worshippers at the very first cathedra..."
...,...,...,...
199995,399996.0,당신은 곧 당신의 조카를 볼 수 있어요.,You can see your nephew soon.
199996,399997.0,나는 당신이 내게 공유한 음악을 들었어요.,I've listened to the music that you shared wit...
199997,399998.0,그 서류는 당신이 공유해 준 것과 달라요.,The document is different from what you shared.
199998,399999.0,당신이 잡지에 광고 낸 것을 봤어요.,I saw your advertisement in the magazine.


In [80]:
data3

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [12]:
data = pd.concat([data3])
data

,대분류,소분류,상황,Set Nr.,발화자,원문,번역문
0,비즈니스,회의,의견 교환하기,1,A-1,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,비즈니스,회의,의견 교환하기,1,B-1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,비즈니스,회의,의견 교환하기,1,A-2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,비즈니스,회의,의견 교환하기,1,B-2,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,비즈니스,회의,의견 교환하기,2,A-1,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...,...,...,...,...,...
99995,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",24999,B-2,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-1,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,B-1,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,여행/쇼핑,쇼핑,"계산/포장/배달 (계산 장소 문의, 계산 오류 등)",25000,A-2,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [13]:
data = data[["원문","번역문"]]
data.rename(columns={"원문":"ko","번역문":"en"},inplace = True)
data

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,ko,en
0,이번 신제품 출시에 대한 시장의 반응은 어떤가요?,How is the market's reaction to the newly rele...
1,판매량이 지난번 제품보다 빠르게 늘고 있습니다.,The sales increase is faster than the previous...
2,그렇다면 공장에 연락해서 주문량을 더 늘려야겠네요.,"Then, we'll have to call the manufacturer and ..."
3,"네, 제가 연락해서 주문량을 2배로 늘리겠습니다.","Sure, I'll make a call and double the volume o..."
4,지난 회의 마지막에 논의했던 안건을 다시 볼까요?,Shall we take a look at the issues we discusse...
...,...,...
99995,"저희가 가격표 배치를 잘못해서 혼동을 드렸나 봐요, 죄송해요.",It seems that we didn't place the price tags c...
99996,"백화점 포인트로 계산하고 싶은데, 가능한가요?",Can I pay using the department store points?
99997,"네, 물론이죠, 전화번호 입력해주시면 됩니다.","Yes, of course, you just need to enter your ph..."
99998,"입력했어요, 전액 백화점 포인트로 결제하고 싶어요.","I entered it, I want to pay it with all the de..."


In [14]:
dataset = Dataset.from_pandas(data)
dataset

Dataset({
    features: ['ko', 'en'],
    num_rows: 100000
})

In [15]:
raw_datasets = Dataset.from_dict({"translation":dataset})
raw_datasets

Dataset({
    features: ['translation'],
    num_rows: 100000
})

In [16]:
from datasets import load_dataset, load_metric


metric = load_metric("sacrebleu")

In [8]:
#데이터셋 개체 자체는 DatasetDict이며, 교육, 검증 및 테스트를 위한 하나의 키를 포함함
#raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 76708
    })
})

In [17]:
#실제 요소에 액세스하려면 먼저 분할을 선택한 다음 인덱스를 제공해야 함
raw_datasets["translation"][0]

{'en': "How is the market's reaction to the newly released product?",
 'ko': '이번 신제품 출시에 대한 시장의 반응은 어떤가요?'}

In [18]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [73]:
show_random_elements(raw_datasets)

,translation
0,"{'en': 'I couldn't finish it because I was away at the weekend, so please let me see your homework in class.', 'ko': '주말에 놀러 갔다 오느라 다 못 했는데, 교실 들어가서 숙제 좀 보여주라.'}"
1,"{'en': 'Cooking shows are getting popular these days.', 'ko': '최근에는 요리하는 방송이 엄청 많은 인기를 얻고 있더라.'}"
2,"{'en': 'I need to use the conference room for a short time. How long will it take?', 'ko': '회의실을 잠깐 써야 해서요. 시간은 얼마나 진행됩니까?'}"
3,"{'en': 'Can I try the tteokbokki here now?', 'ko': '여기 떡볶이 지금 먹어봐도 되는 거예요?'}"
4,"{'en': 'This is my first time here, do you know how to order?', 'ko': '여기는 처음 와 보는데, 주문 어떻게 하는 건지 알아?'}"


In [19]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

In [20]:
#디코딩된 문자열 목록이 되어야 하는 예측 및 레이블을 사용하여 계산 메서드를 호출
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'bp': 1.0,
 'counts': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 4,
 'score': 0.0,
 'sys_len': 4,
 'totals': [4, 2, 0, 0]}

Preprocessing the data

In [21]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [22]:
#if "M2M" in model_checkpoint:
tokenizer.src_lang = "en"
tokenizer.tgt_lang = "ko"

In [97]:
tokenizer("Hello, this one sentence!")

{'input_ids': [128022, 65761, 4, 15911, 13331, 8931, 6226, 30, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [98]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[128022, 65761, 4, 15911, 13331, 8931, 6226, 30, 2], [128022, 36606, 117, 122000, 8931, 6226, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}

In [99]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["안녕하세요!", "잘부탁드려요"]))

{'input_ids': [[128052, 107516, 46446, 30, 2], [128052, 15612, 4159, 64254, 3837, 9068, 5411, 2]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]}


In [23]:
#입력 앞에 특별한 접두사를 붙여야 하는 T5 체크포인트 5개 중 하나를 사용하는 경우
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "translate English to Romanian: "
else:
    prefix = ""

In [24]:
max_input_length = 64
max_target_length = 64
source_lang = "en"
target_lang = "ko"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [103]:
preprocess_function(raw_datasets[:2])

{'input_ids': [[128022, 41797, 66959, 150, 12, 117, 8, 15488, 150, 98953, 16076, 127342, 8251, 128, 118015, 120341, 117007, 28, 1197, 41797, 5, 2], [128022, 3018, 8251, 58275, 120, 8, 8091, 7011, 24, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[128052, 244, 299, 9418, 66959, 150, 12, 1028, 10466, 8230, 526, 79562, 82751, 91029, 21816, 107738, 10633, 1480, 5323, 110366, 8364, 30564, 108193, 10340, 5, 2], [128052, 64783, 11380, 100997, 1903, 7375, 46446, 24, 2]]}

In [25]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/100 [00:00<?, ?ba/s]

model fine tuning

In [26]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [27]:
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    gradient_accumulation_steps= 256,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    push_to_hub=True,
)

In [28]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [29]:
import numpy as np

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

데이터셋과 함께 이 모든 것을 Seq2SeqTrainer에게 전달

In [30]:
tokenized_datasets2 = tokenized_datasets

In [31]:
tokenized_datasets2 = tokenized_datasets2.train_test_split()

In [32]:
tokenized_datasets2

DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 75000
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
})

In [33]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets2["train"],
    eval_dataset=tokenized_datasets2["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/content/m2m100_418M-finetuned-en-to-ko is already a clone of https://huggingface.co/hyerin/m2m100_418M-finetuned-en-to-ko. Make sure you pull the latest changes with `repo.git_pull()`.


이제 train 방법을 호출하는 것만으로 모델을 미세 조정할 수 있다

In [ ]:
#finetuning 전 모델 성능 확인
#trainer.evaluate(max_length=max_target_length)

In [34]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `M2M100ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `M2M100ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 75000
  Num Epochs = 1
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 2048
  Gradient Accumulation steps = 256
  Total optimization steps = 36


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
0,No log,1.946457,6.064400,21.327900


The following columns in the evaluation set  don't have a corresponding argument in `M2M100ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `M2M100ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 25000
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=36, training_loss=2.1800418429904513, metrics={'train_runtime': 4015.7018, 'train_samples_per_second': 18.677, 'train_steps_per_second': 0.009, 'total_flos': 4277989256331264.0, 'train_loss': 2.1800418429904513, 'epoch': 0.98})

In [ ]:
##finetuning 후 모델 성능 확인
#trainer.evaluate(max_length=max_target_length)

In [35]:
#hub 업로드
trainer.push_to_hub()

Saving model checkpoint to m2m100_418M-finetuned-en-to-ko
Configuration saved in m2m100_418M-finetuned-en-to-ko/config.json
Model weights saved in m2m100_418M-finetuned-en-to-ko/pytorch_model.bin
tokenizer config file saved in m2m100_418M-finetuned-en-to-ko/tokenizer_config.json
Special tokens file saved in m2m100_418M-finetuned-en-to-ko/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.33k/1.80G [00:00<?, ?B/s]

Upload file runs/Apr29_07-33-45_ae64ac05f241/1651217654.1661258/events.out.tfevents.1651217654.ae64ac05f241.98…

Upload file runs/Apr29_07-22-33_ae64ac05f241/1651216970.069539/events.out.tfevents.1651216970.ae64ac05f241.98.…

Upload file runs/Apr29_07-19-16_ae64ac05f241/1651216776.1109529/events.out.tfevents.1651216776.ae64ac05f241.98…

...

Upload file runs/Apr29_07-19-16_ae64ac05f241/events.out.tfevents.1651216776.ae64ac05f241.98.2:  91%|######### …

Upload file runs/Apr29_07-11-10_ae64ac05f241/events.out.tfevents.1651216329.ae64ac05f241.98.0:  91%|######### …

Upload file training_args.bin: 100%|##########| 3.11k/3.11k [00:00<?, ?B/s]

To https://huggingface.co/hyerin/m2m100_418M-finetuned-en-to-ko
   480e6fd..54c40d7  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Sequence-to-sequence Language Modeling', 'type': 'text2text-generation'}}
To https://huggingface.co/hyerin/m2m100_418M-finetuned-en-to-ko
   54c40d7..8328815  main -> main



'https://huggingface.co/hyerin/m2m100_418M-finetuned-en-to-ko/commit/54c40d7041c0afd7780121cf03875ee94f2afead'

In [ ]:
#공유
#from transformers import AutoModelForSeq2SeqLM

#model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")